In [ ]:
import ccxt
import schedule
import talib
import numpy as np
import pandas as pd
from datetime import datetime
import time
import pybit
import math


#第一次執行此程式請先至下方https://colab.research.google.com/drive/1xnUcfuuu4FdAImuOkr0IghI-cM53cC7x#scrollTo=_WhNGTvpJGNu&line=1&uniqifier=1 使用硬碟掛載並且把pip install完成才可以使用全部功能
#exchange_id = 'bybit'

#This is for sandbox in testnet
exchange = ccxt.bybit({
    'enableRateLimit': True,
    'apiKey': 'Your API key',
    'secret': 'Your API secret key',
    })
exchange.set_sandbox_mode(True) #使用測試網

#This is for real-time trading
#exchange_class = getattr(ccxt, exchange_id)
#exchange = exchange_class({
#    'apiKey': 'Your API key',
#    'secret': 'Your API secret key',
#})

buy_leverage = 5    #做多槓桿
sell_leverage = 5   #做空槓桿
order = exchange.fetch_ticker('SOLUSDT')

tp_buy = round(order["last"]*(1+0.1/buy_leverage),2)    #做多預計止盈幾%
sl_buy = round(order["last"]*(1-0.1/buy_leverage),2)    #做多預計止損幾%
tp_sell = round(order["last"]*(1-0.1/sell_leverage),2)  #做空預計止盈幾%
sl_sell = round(order["last"]*(1+0.1/sell_leverage),2)  #做空預計止損幾%
params_buy = {
    #"side" : "Buy",
    #"order_type":"Limit",
    "qty":1,
    "take_profit": tp_buy,
    "stop_loss": sl_buy,
    "trailing_stop": 0,
    "time_in_force":"GoodTillCancel"
}
params_sell = {
    #"side" : "Buy",
    #"order_type":"Limit",
    "qty":1,
    "take_profit": tp_sell,
    "stop_loss": sl_sell,
    "trailing_stop": 0,
    "time_in_force":"GoodTillCancel"
}

#exchange.createMarketOrder("APEUSDT","Buy", 5,)
def check_buy_sell_signals(df):
    print("checking for buy and sell signals")
    percent = (df['close']-df['open']) / df['open'] #percent是series 必須設定特定位置為最後一個K棒
    #print(percent)
    if percent[3] >= 0.005:
        exchange.createMarketBuyOrder ("SOLUSDT",0, params_buy)
        print("makeorder")
        
        time.sleep(60*15)
    if percent[3] <= -0.005:
        exchange.createMarketSellOrder ("SOLUSDT",0, params_sell)
        print("makeorder")
        time.sleep(60*15)
    else:
        print("waiting")
        
def run_bot():
    print(f"Fetching new bars for {datetime.now().isoformat()}")
    bars = exchange.fetch_ohlcv('SOLUSDT', timeframe='15m', limit=5)
    df = pd.DataFrame(bars[:-1], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df.insert(6,'percent',value = (df['close']-df['open']) / df['open'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    print(df)
    check_buy_sell_signals(df)
    
    

#print(order["last"])# float type 最新價格 注意order有兩個字典 選第一層就好




#print(exchange.id, exchange.load_markets())
while True:
    run_bot()
    time.sleep(60)




In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install backtesting

In [ ]:
!pip install pybit

In [ ]:
!pip install schedule

In [ ]:
!pip install ccxt

In [ ]:
!pip install talib-binary #use this to download talib!